In [0]:
%pip install -r requirements.txt

In [0]:
%restart_python

In [0]:
import mlflow
import pkg_resources

from mlflow.types.responses import ResponsesAgentRequest

from agent import LLM_ENDPOINT_NAME, AGENT

In [0]:
input_example = {"input": [{"role": "user", "content": "How many Rs in strawberry?"}]}
input_example_extra_params = input_example.copy()
input_example_extra_params["custom_inputs"] = {"temperature": 0.5}
# input_example = ResponsesAgentRequest(input=[{"role": "user", "content": "How many Rs in strawberry?"}])

In [0]:
with mlflow.start_run():
    AGENT.predict(input_example)

In [0]:
with mlflow.start_run():
    AGENT.predict(input_example_extra_params)

In [0]:
resources = AGENT.get_resources()

In [0]:
with open("requirements.txt", "r") as fh:
    requirements = [l.strip() for l in fh.readlines()]

In [0]:
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Workspace/Users/stuart.lynn@databricks.com/u2m-experiment")

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        input_example=input_example,
        resources=resources,
        extra_pip_requirements=requirements
    )

In [0]:
loaded_model = mlflow.pyfunc.load_model(f"runs:/{logged_agent_info.run_id}/agent")
loaded_model.predict(input_example)

In [0]:
loaded_model.predict(input_example_extra_params)

In [0]:
mlflow.models.predict(logged_agent_info.model_uri, input_example, env_manager="uv")

In [0]:
mlflow.models.predict(logged_agent_info.model_uri, input_example_extra_params, env_manager="uv")

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS stuart;
CREATE SCHEMA IF NOT EXISTS stuart.authtest;

In [0]:
mlflow.set_registry_uri("databricks-uc")

registered_model = mlflow.register_model(
    model_uri=logged_agent_info.model_uri,
    name="stuart.authtest.agent"
)

In [0]:
import databricks.agents

endpoint_name = "custom-agent-endpoint"

databricks.agents.deploy(
    model_name=registered_model.name,
    model_version=registered_model.version,
    scale_to_zero=True,
    endpoint_name=endpoint_name,
)